In [71]:
from transformers import pipeline
from opencage.geocoder import OpenCageGeocode
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
import re
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


ner = pipeline("ner", grouped_entities=True)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/sds/geospatial-map/lib/pyt

In [72]:
# ZOOM IN 
#commands = [
#     "zoom in to Delhi",
#     "magnify to Bangalore",
#     "zoom to Delhi"    
# ]

In [76]:
#ZOOM  OUT
# commands = [
#     "zoom out to Delhi",
#     "minimize to bangalore",
#     "get me out of Delhi"
# ]

In [78]:
#SEARCH
# commands = [
#     "search for Delhi",
#     "show me Bangalore",
#     "find Delhi"
# ]


In [80]:
#Directions

# commands = [
#     "direct me to Delhi",
#     "show me the way to Bangalore",
#     "How to reach Delhi?"
# ]

In [84]:
# #clear

# commands= [
#     "Clear Map",
#     "Go to home page",
#     "Reset the map"
# ]

In [90]:
#MARKING

# commands= [
#     "Choose Delhi",
#     "Pick bangalore",
# ]


In [96]:
#SWITCH
commands = [
    "change maps",
    "swap maps"
]

In [73]:

# Initialize with your API key
key = '0188553be00944a0954333a31eae57d9'
geocoder = OpenCageGeocode(key)

In [64]:
#CO-ORDINATES

In [74]:
for command in commands:
    print(f"\nProcessing command: '{command}'")
    ner_results = ner(command)
    
    # Loop through NER results and handle locations
    for entity in ner_results:
        print(f"Entity: {entity['word']}, Type: {entity['entity_group']}, Confidence: {entity['score']:.4f}")
        
        # Check if the entity type is a location
        if entity['entity_group'] == 'LOC':
            result = geocoder.geocode(entity['word'])
            if result:
                location = result[0]['geometry']
                print(location['lat'], location['lng'])

        else:
            print(f"Skipping non-location entity: {entity['word']}")



Processing command: 'zoom in to Delhi'
Entity: Delhi, Type: LOC, Confidence: 0.9995
28.6273928 77.1716954

Processing command: 'magnify to Bangalore'
Entity: Bangalore, Type: LOC, Confidence: 0.9986
12.9767936 77.590082

Processing command: 'zoom to Delhi'
Entity: Delhi, Type: LOC, Confidence: 0.9994
28.6273928 77.1716954


In [66]:
# def remove_ner_entities(command):
#     ner_results = ner(command)
#     # Extracting the entities
#     entities = [entity['word'] for entity in ner_results]
#     # Removing entities from command
#     for entity in entities:
#         command = command.replace(entity, '').strip()
#     return command


In [69]:
# cleaned_commands = []    
# # Process commands
# for command in commands:
#     cleaned_command = remove_ner_entities(command)
#     cleaned_command = re.sub(r'\bzoom in\b', 'zoom_in', cleaned_command, flags=re.IGNORECASE)    # Store cleaned command
#     cleaned_command = re.sub(r'\bzoom out\b', 'zoom_out', cleaned_command, flags=re.IGNORECASE)    # Store cleaned command

#     cleaned_commands.append(cleaned_command)

#     print(f"Original Command: '{command}' => Cleaned Command: '{cleaned_command}'")

In [97]:


# Load SBERT model for sentence embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define target actions
target_actions = {
    "zoom_in": ["zoom in", "magnify", "enlarge"],
    "zoom_out": ["zoom out", "minimize"],
    "directions": ["find directions", "navigate", "directions"],
    "search": ["search", "look for"],
    "reset": ["reset","clear"],
    "mark": ["mark","select","set pin","highlight"],
    "switch": ["switch","change"]
    #exit?
}

# Get embeddings for target action phrases
target_embeddings = {key: model.encode(phrases) for key, phrases in target_actions.items()}

def get_best_match(command):
    command_embedding = model.encode(command)
    best_match = None
    highest_similarity = 0.0
    
    for action, embeddings in target_embeddings.items():
        # Calculate similarity for each target action's embedding list
        similarities = [cosine_similarity([command_embedding], [embed])[0][0] for embed in embeddings]
        max_similarity = max(similarities)
        
        if max_similarity > highest_similarity:
            highest_similarity = max_similarity
            best_match = action
    
    return best_match, highest_similarity

for command in commands:
    best_match, similarity = get_best_match(command)
    print(f"Command: '{command}' => Keyword: '{best_match}' (Similarity: {similarity:.4f})")


Command: 'change maps' => Keyword: 'switch' (Similarity: 0.5237)
Command: 'swap maps' => Keyword: 'switch' (Similarity: 0.4664)
